# Examples on using pydggapi endpoints

This notebook illustrates some sample uses of the OGC DGGS-API endpoint. The API endpoints can be categorised into three categories.

**DGGS API endpoints:**

The queries below work on all datasets defined in the `dggs_api_config.json` file. The API only returns results if the zones of the query exist in any of the collections. The API endpoints are organised by conformance class.

- Core conformance class
  - (`/`) : landing page
  - (`/dggs`) : query supported dggrs 
  - (`/dggs/<dggrs_id>`) : query dggrs definition 
  - (`/dggs/<dggrs_id>/zones/<zone_id>`) : query zone info 
- Zone Query conformance class
  - (`/dggs/<dggrs_id>/zones`) : listing zones of a specific bounding box.
- Zone Data Retrieval conformance class 
  - (`/dggs/<dggrs_id>/zones/<zone_id>/data`) : get data for a specific zone 

**Collection DGGRS API endpoints:**

Apart from query zones for all datasets, the above queries can be used for a specific collection:
- (`/collections`): list all collections that are published
- (`/collections/<collection_id>`): list the meta data of the specific collection
- (`/collections/<collection_id>/queryables`): list the attributes of the specific collection
- (`/collections/<collection_id>/dggs/...`): Only query on the specific collection with the DGGS API endpoints from above.

**Tiles API for collection:**
The API supports returning collection data in vector tile format. This enables applications such as QGIS to visualise the data as a layer easily.

- (`/tiles-api/<collection_id>.json`) : It returns the tiles json of the collection
- (`/tiles-api/<collection_id>/{z}/{x}/{y}`): The tiles source url

## Setup
All api endpoints work with HTTP GET requests. In this example, we primarily use the requests library to perform HTTP queries. 

The API returns JSON by default; it also supports returning various data types, which we will demonstrate in the upcoming section.

In [1]:
import requests
# to display the return json
import json
from IPython.display import display, JSON, GeoJSON

# define the api root url
api_root = "http://127.0.0.1:8000"
dggs_api_root=f"{api_root}/dggs-api"

## DGGS API endpoints

### DGGS API Core conformance class

#### **landing page (`<dggs_api_root>/`)**

The landing page provides links to the API definition.

In [2]:
# send the query, and parse the return to json
query_return = requests.get(dggs_api_root).json()
# display it nicely with IPython display module
JSON(query_return)

<IPython.core.display.JSON object>

#### **Query supported dggrs (`<dggs_api_root>/dggs`)**

This query lists all DGGRs supported by the API instance in the list `dggrs`.

In [3]:
query_return = requests.get(dggs_api_root+'/dggs').json()
JSON(query_return)

<IPython.core.display.JSON object>

#### **Query dggrs definition (`<dggs_api_root>/dggs/<dggrs_id>`)**
This query returns the definition and details of the dggrs

In [4]:
query_return = requests.get(dggs_api_root+'/dggs/igeo7').json()
display(JSON(query_return))

<IPython.core.display.JSON object>

#### **Query zone info (`<dggs_api_root>/dggs/<dggrs_id>/zones/<zone_id>`)**
This query returns the zone's properties (zone's centroid, zone's geometry and area etc.)

In [5]:
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones/0001022010').json()
JSON(query_return)

<IPython.core.display.JSON object>

### DGGS API Zone Query conformance class

#### **Zone Query (`<dggs_api_root>/dggs/<dggrs_id>/zones`)**

This query accepts the following query parameters: 

- [bbox](#bbox:-Return-a-list-of-zones-that-is-contains-in-the-specified-bounding-box-at-refinement-level-6)
  - Bounding box of the query. The bounding box is provided as four coordinates, in the order of minx,miny,maxx and maxy
- [zone-level](#bbox:-Return-a-list-of-zones-that-is-contains-in-the-specified-bounding-box-at-refinement-level-6)
  - For specifying a level at which to return a list of DGGRS zones using a zone_level query parameter.
- [compact-zone](#bbox:-Return-a-list-of-zones-that-is-contains-in-the-specified-bounding-box-at-refinement-level-6)
  - For specifying whether to retrieve a list of DGGRS zones using a compact_zones query parameter.
- [parent-zone](#parent-zone:-Return-a-list-of-zones-at-refinement-level-6-with-a-parnet-zone-at-refinement-level-5)
  - For specifying a parent zone within which to restrict zone listing using a parent_zone query parameter.
- [geometry](#geometry:-Change-the-return-type-to-geojson-and-return-zone-centroid-instead-of-zone-region)
  - Specify the return geometry (`zone-centroid` or `zone-region`), defaults to `zone-region` 
- [filter](#filter:-Using-CQL2-string-to-filter-records-for-all-collections)
  - A CQL2 filter string
- [datetime](#datetime:-filter-collections-with-datetime-interval)
  - Specify the datetime interval

Users can specify the return type in the HTTP header or using the `format query string`. The zone query supports the following return types, defualts to `json`: 
- `?f=json` or set `application/json` in the http header 
- `?f=geojson` or `application/geo+json` in the http header


##### **`bbox`: Return a list of zones that is contains in the specified bounding box at refinement level 6**

In [6]:
# To query the zones list of a bounding box at refinement level 6 with compact zone set to off 
bbox = "25.329803558251513,57.99111013411327,27.131561370751513,58.634545591972696"
zone_level=6
compact_zone=False
# send the query 
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'bbox': bbox, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone}) 
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

##### **`parent-zone`: Return a list of zones at refinement level 6 with a parnet zone at refinement level 5**

In [25]:
# To query the zones list at refinement level 6 using a parent zone at refinement level 5  with compact zone set to off 
# define the parent zone at refinement level 5
parent_zone='0001613'
zone_level=6
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'parent-zone': parent_zone, 'zone-level': zone_level,
                                                                  'compact-zone': False}) 
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

##### **`&f=geojson`: Change the return type to geojson**

In [8]:
parent_zone='0001026' 
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'parent-zone': parent_zone, 'zone-level': zone_level,
                                                                  'compact-zone': False, 'f': 'geojson'}) 
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

##### **`geometry`: Change the return type to geojson and return zone centroid instead of zone region**

In [9]:
parent_zone='0001026' 
compact_zone=False
geometry = 'zone-centroid'
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'parent-zone': parent_zone, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone,'geometry': geometry,
                                                                  'f': 'geojson'}) 
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

##### **`filter`: Using CQL2 string to filter records for all collections**
[Common Query language (CQL)](https://docs.ogc.org/is/21-065r2/21-065r2.html#_terms_definitions_symbols_and_abbreviated_terms)

[CQL Expression](https://docs.eoxserver.org/en/stable/users/services/cql.html)

Before applying the CQL2 filter feature, users need to know which attributes are available for filtering. 

Users can use the `collections/<collection_id>/querables` to find out those attributes. Find out more details of the collection api [here](#Query-the-collection-meta-data-(<dggs_api_root>/collections/<collection_id>))

In [10]:
collection_id='suitability_hytruck_parquet_local' 
query_return = requests.get(dggs_api_root+f'/collections/{collection_id}/queryables')
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

From the list of properties, users can find the attribute name in the form of `<collection_id>.<attribute_name>`. When using the CQL2 filter, only the attribute name is needed. In this case, the filter works on multiple collections if those collections share the same attribute name.

In [11]:
bbox = "25.329803558251513,57.99111013411327,27.131561370751513,58.634545591972696"
zone_level=7
compact_zone=False
# looking for zones that the attribute modelled_slope_band_1 >=6
cql_filter = "modelled_slope_band_1 >= 6"
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'bbox': bbox, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone, 'filter': cql_filter}) 
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

In [12]:
bbox = "25.329803558251513,57.99111013411327,27.131561370751513,58.634545591972696"
zone_level=7
compact_zone=False
# demonstrate the filter by changing the value to >=10
cql_filter = "modelled_slope_band_1 >= 10"
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'bbox': bbox, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone, 'filter': cql_filter}) 
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

If no records are found in all collections, the API returns empty content (HTTP status code 204).

In [13]:
bbox = "25.329803558251513,57.99111013411327,27.131561370751513,58.634545591972696"
zone_level=7
compact_zone=False
# demonstrate the filter by changing the value to >=10
cql_filter = "modelled_slope_band_1 > 10"
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'bbox': bbox, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone, 'filter': cql_filter})

if (query_return.status_code != 204):
    query_return = query_return.json()
    display(JSON(query_return))
else:
    print("empty return")

empty return


In [14]:
bbox = "25.329803558251513,57.99111013411327,27.131561370751513,58.634545591972696"
zone_level=7
compact_zone=False
# A more complex CQL expression
cql_filter = "(survey_suitability_value > 2 AND survey_suitability_value < 4) \
               AND modelled_slope_band_1 >=10"
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'bbox': bbox, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone, 'filter': cql_filter})

if (query_return.status_code != 204):
    query_return = query_return.json()
    display(JSON(query_return))
else:
    print("empty return")

<IPython.core.display.JSON object>

##### **`datetime`: filter collections with datetime interval**

The format of the datetime follows the RFC 3339, section 5.6.
Users can specify a specific date or an interval with the following syntax:

- To query data on a specific date: `datetime=<datetime>` 
- To query data within an interval: `datetime=<datetime>/<datetime>`
- To query data after the specified datetime `datetime=<datetime>/..`
- To query data before the specified datetime `datetime=../<datetime>`

In [15]:
bbox = "-98.46911119,48.62133077,-95.99420856, 50.77510588"
zone_level=5
compact_zone=False
# Datetime in YYYY-MM-DD format
datetime = "2025-09-11/.."
query_return = requests.get(dggs_api_root+'/dggs/igeo7/zones', params={'bbox': bbox, 'zone-level': zone_level,
                                                                  'compact-zone': compact_zone, 'datetime': datetime})

if (query_return.status_code != 204):
    query_return = query_return.json()
    display(JSON(query_return))
else:
    print("empty return")

<IPython.core.display.JSON object>

### DGGS API Zone data retrieval conformance class 

#### **Zone data retrieval (`<dggs_api_root>/dggs/<dggrs_id>/zones/<zone_id>/data`)**

This query accepts the following query parameters: 

- [zone-depth](#zone-depth:-Return-the-data-associated-with-the-zone_id-at-the-zone_id's-refinement-level-in-DGGS-JSON-format)
  - For specifying a level at which to return a list of DGGRS zones using a zone_level query parameter.
- geometry
  - Specify the return geometry (`zone-centroid` or `zone-region`), defaults to `zone-region` 
- filter
  - A CQL2 filter string
- datetime
  - Specify the datetime interval

Users can specify the return type in the HTTP header or using the `format query string`. The zone query supports the following return types, defualts to `json`: 
- `?f=json` or set `application/json` in the http header (default)
- `?f=ubjson` or `application/ubjson` in the http header 
- `?f=geojson` or `application/geo+json` in the http header
- `?f=zarr` or `application/zarr+zip` in the http header

##### **`zone-depth`: Return the data associated with the `zone_id` at the `zone_id`'s refinement level in DGGS-JSON format**

In [16]:
zone_id = '000102062'
zone_depth = "0"
query_return = requests.get(dggs_api_root+f'/dggs/igeo7/zones/{zone_id}/data', params={'zone-depth': zone_depth})

if (query_return.status_code != 204):
    query_return = query_return.json()
    display(JSON(query_return))
else:
    print("empty return")

<IPython.core.display.JSON object>

Users can query data at a finer refinement level using the `zone-depth` parameters relative to the refinement level of the `zone_id`

In [17]:
zone_id = '000102062'
# The actual refinement level for the query is at level 9, zone_id refinement level + zone-depth
zone_depth = "2" 
query_return = requests.get(dggs_api_root+f'/dggs/igeo7/zones/{zone_id}/data', params={'zone-depth': zone_depth})

if (query_return.status_code != 204):
    query_return = query_return.json()
    display(JSON(query_return))
else:
    print("empty return")

<IPython.core.display.JSON object>

Users can also retrieve data with 2 consecutive refinement levels

In [18]:
zone_id = '0001020'
# 2 consecutive refinement levels (5, 6)
zone_depth = "0-1"
return_type = 'geojson'
query_return = requests.get(dggs_api_root+f'/dggs/igeo7/zones/{zone_id}/data', params={'zone-depth': zone_depth, 'f': return_type})

if (query_return.status_code != 204):
    query_return = query_return.json()
    display(JSON(query_return))
else:
    print("empty return")

<IPython.core.display.JSON object>

### Collections endpoints

#### **List all collections (`<dggs_api_root>/collections`)**

In [19]:
query_return = requests.get(dggs_api_root+f'/collections')
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

#### **Query the collection meta data (`<dggs_api_root>/collections/<collection_id>`)**

In [20]:
collection_id = "suitability_hytruck_parquet_local"
query_return = requests.get(dggs_api_root+f'/collections/{collection_id}')
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

#### **Query the attributes of the collection(`<dggs_api_root>/collections/<collection_id>/queryables`)**

In [21]:
collection_id = "suitability_hytruck_parquet_local"
query_return = requests.get(dggs_api_root+f'/collections/{collection_id}/queryables')
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

## Tiles API endpoints

### **Query the collection tiles json (`<api-root>/tiles-api/<collection_id>.json`)**

In [22]:
collection_id = "suitability_hytruck_parquet_local"
query_return = requests.get(api_root+f'/tiles-api/{collection_id}.json')
query_return = query_return.json()
JSON(query_return)

<IPython.core.display.JSON object>

### **Query data with the tiles source url (`<api-root>/tiles-api/<collection_id>/{z}/{x}/{y}`)**

The endpoint supports the `relative_depth` query parameter which allow users to retrieve data at a finer refinement level.

Here is the example of the url: `<api-root>/tiles-api/<collection_id>/{z}/{x}/{y}?relative_depth=1`